# toorPIAによる分析

以下では、toorPIAによる時系列データの典型的分析の内容を示す。

ここでは、toorPIAという手法を学ぶというより、「**時系列データに対する観測時間窓の幅(window size)の影響**」についての感覚をつかんでほしい。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# サンプルデータの読み込みと確認

# データの読み込み
rawDataPath = 'rawdata/testdata_for_toor.csv'
alldata = pd.read_csv(rawDataPath, encoding="UTF8")

# labelカラムの値をlabels配列に格納
labels = alldata['label'].values

# プロット描画準備
fig, ax = plt.subplots(12, 5, figsize=(40, 100))

ci = 0
for column_label in alldata.columns:
    fig_row = ci // 5
    fig_col = ci % 5
    if column_label == 'id':
        continue
    if column_label == 'label':
        break
    ci += 1

    xBase, yBase = alldata['id'].values, alldata[column_label].values
    ax[fig_row, fig_col].set_xlim(min(xBase), max(xBase))
    ax[fig_row, fig_col].set_ylim(min(yBase), max(yBase))
    ax[fig_row, fig_col].set_title(f'column={column_label}')

    if (ci >= 1 and ci <= 10) or (ci >= 31 and ci <= 60):
        ax[fig_row, fig_col].plot(xBase, yBase, c="grey", alpha=0.5)
        ax[fig_row, fig_col].scatter(xBase, yBase, c="blue", s=1.0)
        continue
    elif ci >= 11 and ci <= 20:
        ax[fig_row, fig_col].plot(xBase, yBase, c="grey", alpha=0.5)
        ax[fig_row, fig_col].scatter(xBase, yBase, c="blue", label='normal', s=1.0)
        idx = np.where(labels=='warm-up')
        ax[fig_row, fig_col].scatter(xBase[idx], yBase[idx], c="green", label='worm up', s=1.0)
        ax[fig_row, fig_col].legend()
        continue
    elif ci >= 26 and ci <= 30:
        ax[fig_row, fig_col].scatter(xBase, yBase, c="blue", label='normal', s=1.0)
        continue

        

    ax[fig_row, fig_col].plot(xBase, yBase, c="grey", alpha=0.5)
    idx = np.where(labels=='warm-up')
    ax[fig_row, fig_col].scatter(xBase[idx], yBase[idx], c="green", label='worm up', s=1.0)
    idx = np.where(labels=='normal')
    ax[fig_row, fig_col].scatter(xBase[idx], yBase[idx], c="blue", label='normal', s=1.0)
    idx = np.where(labels=='abnormal1')
    ax[fig_row, fig_col].scatter(xBase[idx], yBase[idx], c="red", label='abnormal1', s=1.0)
    idx = np.where(labels=='abnormal2')
    ax[fig_row, fig_col].scatter(xBase[idx], yBase[idx], c="orange", label='abnormal2', s=1.0)
    ax[fig_row, fig_col].legend()

In [ ]:
# toorPIAによる分析

import toorpia.utils as tp

In [ ]:
# WS=1でマップ表示
params = tp.load_params('analysis_ws1')
print("ws: " , params['window_size'])
x, y = tp.create_basemap(params, force=True)

ws=1の結果においては、c26〜c30カラムの振動(3つの状態)を反映したマップになっている

In [ ]:
# WS=30でマップ表示
params = tp.load_params('analysis_ws30')
print("ws: " , params['window_size'])
x, y = tp.create_basemap(params, force=True)

ws=30にすると、ws=1の時に観測されたc26〜c30の振動状態が消失していることがわかる

In [ ]:
# WS=200でマップ表示
params = tp.load_params('analysis_ws200')
print("ws: " , params['window_size'])
x, y = tp.create_basemap(params, force=True)

ws=200にすると、データが時間軸に沿ってひも状に並ぶようになってきた

In [ ]:
# WS=600でマップ表示
params = tp.load_params('analysis_ws600')
print("ws: " , params['window_size'])
x, y = tp.create_basemap(params, force=True)

ws=600にすると、更にデータが時間軸に沿ってひも状に。これによって、本データの時間依存性をわかりやすく捉えることができる

## まとめ

以上のように、データをどのような時間幅について解析するかによって、観測される事象が変化する。

解析の目的が、細かい状態変化を捉えることにあるのであればwsはできるだけ小さい方がよく、一方、粗視化によって全体のトレンドを把握する場合にはwsを適当に大きくとってやる必要がある。